<a href="https://colab.research.google.com/github/girija2204/DLTopics/blob/master/PyTorch/Simple_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
train_data=datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True)
test_data=datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)

In [ ]:
train_loader=DataLoader(dataset=train_data,batch_size=64,shuffle=True)
test_loader=DataLoader(dataset=test_data,batch_size=64,shuffle=True)

In [ ]:
DEVICE=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class NN(nn.Module):
  def __init__(self,input_size,num_classes):
    super(NN,self).__init__()
    self.fc1=nn.Linear(in_features=input_size,out_features=60)
    self.fc2=nn.Linear(in_features=60,out_features=num_classes)
  
  def forward(self,x):
    x=F.relu(self.fc1(x))
    x=self.fc2(x)
    return x

In [ ]:
input_size=784
output_size=10
num_epochs=20
learning_rate=0.001

In [ ]:
model=NN(input_size=input_size,num_classes=output_size).to(DEVICE)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
  for batch_idx, (data, target) in enumerate(train_loader):
    data=data.to(DEVICE)
    target=target.to(DEVICE)

    data=data.reshape(data.shape[0],-1)

    scores=model(data)
    loss=criterion(scores,target)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

In [ ]:
def check_accuracy(loader,model):
  if loader.dataset.train:
    print(f"Checking accuracy for training set")
  else:
    print(f"Checking accuracy for testing set")
  num_correct=0
  num_samples=0
  model.eval()
  with torch.no_grad():
    for data,target in loader:
      data=data.to(DEVICE)
      target=target.to(DEVICE)
      data=data.reshape(data.shape[0],-1)
      scores=model(data)
      _,predictions=scores.max(1)
      num_correct+=(predictions==target).sum()
      num_samples+=predictions.size(0)
    print(f"Got {num_correct}/{num_samples} accuracy: {(num_correct/num_samples)*100:.2f}")
  model.train()

In [ ]:
check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

Checking accuracy for training set
Got 59733/60000 accuracy: 99.56
Checking accuracy for testing set
Got 9720/10000 accuracy: 97.20
